In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df_pitch_subs = pd.read_csv('pitching_subs.csv')
df_next_pitcher = df_pitch_subs.shift(-1).reset_index()
df_pitch_subs.tail(50)
df_pitch_subs['made_save'] = None

In [3]:
for t1, t2 in zip(df_pitch_subs.iterrows(), df_next_pitcher.iterrows()):
    cur_index = t1[0]
    cur_row = t1[1]
    next_row = t2[1]
    made_save = None
    
    b = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
    if cur_row['GAME_ID'] == next_row['GAME_ID'] and cur_row['HOME_TEAM'] == next_row['HOME_TEAM']:
        # in same game, and same team.
        if cur_row['HOME_TEAM'] == 1:
            # home team
            blew_lead = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
            if blew_lead:
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or (inn_diff >= 3) or (cur_row['HOME_SCORE_CT'] <= cur_row['AWAY_SCORE_CT'] + 2 + sum([cur_row['onFirst'], cur_row['onSecond'], cur_row['onThird']])):
                    made_save = False
                
            
        elif cur_row['HOME_TEAM'] == 0:
            # away team
            blew_lead = next_row['HOME_SCORE_CT'] >= next_row['AWAY_SCORE_CT']
            if blew_lead:
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['AWAY_SCORE_CT'] - cur_row['HOME_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or (inn_diff >= 3) or (cur_row['AWAY_SCORE_CT'] <= cur_row['HOME_SCORE_CT'] + 2 + sum([cur_row['onFirst'], cur_row['onSecond'], cur_row['onThird']])):
                    made_save = False
    
    
    elif cur_row['PIT_ID'] == cur_row['SAVE_PIT_ID']:
            made_save = True
               

    df_pitch_subs.iloc[cur_index, df_pitch_subs.columns.get_loc('made_save')] = made_save
    
    
df_pitch_subs.head(50)


,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID,made_save
0,ANA201004050,jepsk001,7,1,0,3,4,0,0,0,fuenb001,None
1,ANA201004050,rodnf001,8,1,0,3,4,0,0,0,fuenb001,None
2,ANA201004050,fuenb001,9,1,0,3,6,0,0,0,fuenb001,True
3,ANA201004050,craij001,5,0,2,3,4,1,0,1,fuenb001,False
4,ANA201004050,mijaj001,7,0,1,3,4,0,0,0,fuenb001,False
5,ANA201004050,neshp001,8,0,0,3,6,0,0,0,fuenb001,None
6,ANA201004060,palmm001,6,1,0,5,3,0,0,0,raucj001,False
7,ANA201004060,stokb001,8,1,0,5,3,0,0,0,raucj001,None
8,ANA201004060,duenb001,7,0,2,5,3,0,1,0,raucj001,None
9,ANA201004060,guerm001,8,0,0,5,3,0,0,0,raucj001,None


In [4]:
def save_sit(row):
    if row['HOME_TEAM'] == 1 and row['HOME_SCORE_CT'] > row['AWAY_SCORE_CT']:
        return True
    elif row['HOME_TEAM'] == 0 and row['HOME_SCORE_CT'] < row['AWAY_SCORE_CT']:
        return True
    else:
        return False

df_pitch_subs['SAVE_SIT'] = df_pitch_subs.apply(save_sit, axis=1)

In [5]:
df_pitch_subs = df_pitch_subs[df_pitch_subs['SAVE_SIT'] == True].reset_index()
df_pitch_subs.head(50)

,index,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID,made_save,SAVE_SIT
0,0,ANA201004050,jepsk001,7,1,0,3,4,0,0,0,fuenb001,None,True
1,1,ANA201004050,rodnf001,8,1,0,3,4,0,0,0,fuenb001,None,True
2,2,ANA201004050,fuenb001,9,1,0,3,6,0,0,0,fuenb001,True,True
3,8,ANA201004060,duenb001,7,0,2,5,3,0,1,0,raucj001,None,True
4,9,ANA201004060,guerm001,8,0,0,5,3,0,0,0,raucj001,None,True
5,10,ANA201004060,raucj001,9,0,0,5,3,0,0,0,raucj001,True,True
6,13,ANA201004070,guerm001,8,0,0,4,1,0,0,0,raucj001,None,True
7,14,ANA201004070,raucj001,9,0,0,4,1,0,0,0,raucj001,True,True
8,18,ANA201004080,neshp001,6,0,1,3,1,1,1,0,None,None,True
9,19,ANA201004080,mijaj001,7,0,0,3,1,0,0,0,None,None,True


In [7]:
df_blown_save = df_pitch_subs[df_pitch_subs['made_save'] == False]
df_blown_save.head(50)

,index,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID,made_save,SAVE_SIT
15,27,ANA201004100,shies002,8,1,0,2,3,0,0,0,None,False,True
16,32,ANA201004110,jepsk001,7,1,0,2,3,0,1,1,rosst001,False,True
26,48,ANA201004210,fuenb001,9,1,0,2,3,0,0,0,valvj001,False,True
48,99,ANA201005100,fuenb001,9,1,0,1,4,0,0,0,None,False,True
85,184,ANA201006250,rodnf001,8,1,0,2,3,0,0,0,corpm001,False,True
100,216,ANA201007020,fuenb001,9,1,0,0,1,1,0,0,sorij001,False,True
123,267,ANA201007300,harrm001,4,0,0,7,6,0,1,1,fuenb001,False,True
160,346,ANA201009080,rodnf001,9,1,0,2,3,0,0,0,None,False,True
161,358,ANA201009100,rodnf001,9,1,0,1,3,0,0,0,None,False,True
178,411,ANA201009270,jamej001,7,0,2,4,3,1,1,0,rodnf001,False,True


In [ ]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'HOME_TEAM', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(5)

In [ ]:

df_next_pitcher[['GAME_ID', 'PIT_ID', 'INN_CT', 'HOME_TEAM', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(100)